# Determinisitc Policy Gradient

- Expected gradient of the action value function 형태이다.
- Policy variance가 0으로 수렴할 경우, DPG는 Stochastic Policy Gradient와 동일해진다.
  - 따라서, Policy gradient에 기법을 DPG에 적용가능하다.
- 효율적인 exploration을 위해서, model-free, off-policy actor critic 알고리즘이 제안되었다.
  - action-value function approximator를 사용해서 발생하는 bias를 막기 위해서 compatibility condition을 제공한다.

- DPG는 SPG보다 성능이 좋다.
  - high dimensional action spaces에서 성능 향상이 크다.
      - SPG의 policy gradient는 state와 action spaces 모두에 대해서, DPG의 policy gradient는 state spaces에 대해서 평균을 취한다.
      - 결과적으로, action spaces의 차원이 커질수록 data efficiency가 높은 DPG의 학습이 더 잘 이루어진다.
  - 기존 방식보다 computation이 많지 않다.

## 개요

- DQN 이후, Policy gradient가 중요한 알고리즘으로 생각되고 있다. 하지만 여기서 Actor critic 알고리즘이 기본이 된다.
- Policy gradient는 Policy gradient Theorem을 근거로 발전하고 있다.
- Stochastic policy gradient theorem은 policy를 state space와 action space 두 개의 확률분포를 고려한다.
- 하지만, Deterministic policy gradient는 policy를 action space에서 action은 고정하고 policy를 update할 수 있다는 것을 보여준다.

### Policy gradient theorem

- 우리는 likelihood ratios의 방법을 이용해서 score function을 아래와 같이 나타낼 수 있다.<br><br>
$$ \begin{split} \triangledown_{\theta} \pi_{\theta}(s, a) &= \pi_{\theta}(s, a) \frac{\triangledown_{\theta}\pi_{\theta}(s, a)}{\pi_{\theta}(s, a)} \\
&= \pi_{\theta}(s, a) \triangledown_{\theta} log \pi_{\theta}(s,a) \end{split} $$<br>
- 여기서 objective function의 gradient를 구하면,<br><br>
$$ \mathbb{E}_{\pi_{\theta}}[\triangledown_{\theta} log \pi_{\theta}(s, a)r] $$

- 위 objective function의 gradient에 있는 \\(r\\)을 Q function으로 대체할 수 있다는 이론이다. 최종 식은<br><br>

$$ \triangledown_\theta J(\theta) = \mathbb{E}_{\pi_{\theta}}[\triangledown_{\theta} log \pi_{\theta}(s, a)Q^{\pi_{\theta}}(s, a)] $$

- 하지만, 위 식은 stochastic policy gradient로써 policy자체가 stochastic하다. 따라서 policy를 두가지 방법으로 표현할 수 있다.
  - Softmax policy
  - Gaussian policy

## Stochastic Actor-critic algorithm

- actor와 critic이 번갈아가면서 동작하며 stochastic policy를 최적화한다.
- actor는 위 policy gradient theorem으로 정의된 objective function을 통해서 parameter를 update한다.

## Actor

- \\(Q^{\pi}(s, a)\\)를 근사한 \\(Q^w(s, a)\\)를 이용해서 stochastic policy gradient를 ascent하는 방향으로 policy parameter \\(\theta\\)를 update함으로써 stochastic policy를 개선한다.<br><br>
$$ \nabla_{\theta}J(\pi_{\theta}) = E_{s \sim \rho^{\pi}, a \sim \pi_{\theta}}[\nabla_{\theta}\log\pi_{\theta}(a|s)Q^{w}(s,a)] $$

## Critic

- SARSA나 Q-learning같은 Temporal-difference(TD) learning을 이용해서 action-value function의 parameter \\(w\\)를 update함으로써 \\(Q^w(s, a)\\)가 \\(Q^{\pi}(s, a)\\) 유사하도록 한다.
- 실제 값 \\(Q^{\pi}(s, a)\\)을 사용하지 않고 근사값 \\(Q^{w}(s, a)\\)를 사용하게 되면 bias가 발생한다. 하지만 compatible condition에 부합하는 근사값을 사용하면 bias가 없다.

## Off-policy Actor-Critic

- Distinct behavior policy \\(\beta(a|s) ( \neq \pi_{\theta}(a|s) )\\)로 부터 sampling된 경로를 이용한 actor-critic
- off-policy actor-critic은 \\(\beta\\)에 의해 sampling 된 trajectory를 이용해서 policy \\(\pi\\)를 근사하는 것이기 때문에 importance sampling이 필요함.

- True policy(\\(\pi_{\theta}(a|s)\\))와 behavior policy(\\(\beta_{\theta}(a|s)\\))의 비율을 importance sampling ratio라 부른다.
- 따라서 off-policy actor-critic의 objective function은 아래와 같이 표현된다.<br><br>
$$ \triangledown_{\theta} J_{\beta}(\pi_{\theta}) = \mathbb{E}_{s \sim \rho^\beta, a \sim \beta_{\theta}} \left[ \frac{\pi_{\theta}(a|s)}{\beta_{\theta}(a|s)} \triangledown_{\theta} log \pi_{\theta}(a|s) Q^w(s,a) \right] $$

## DPG

- Generalized policy iteration
  - policy evaluation과 policy improvement를 한 번씩 번갈아 가면서 실행하는 policy iteration
  - value function은 optimal에 수렴함.
- Policy evalutation, action-value function \\(Q^{\pi}(s, a)\\)나 \\(Q^{\mu}(s, a)\\)를 estimate 하는 것이다.
- Policy improvement,
  - estimated action-value function에 따라 정책을 update한다.
  - action-value function에 대한 greedy max를 사용하게 되는데, continuous action spaces가 되면 계산량이 많이 늘어난다.
  - 따라서, policy gradient를 사용한다.

## Deterministic Actor Critic Algorithms

- DPG theorem에서는 action space에 대한 적분항이 사라졌다. chain rule과 비슷하게 계산이 적용되어 action(deterministic action) space에서 Q value의 gradient와 action network로 표현되는 action에 대한 확률 분포의 gradient 곱으로 표현가능하다는 것이 DPG의 핵심이다.
- objective function은 아래와 같다.<br><br>
$$ \begin{split} \triangle_{\theta} J(\mu_{\theta}) &= \int_{\mathcal{S}} \rho^{\mu}(s) \triangledown_{\theta} \mu_{\theta}(s) \triangledown_a Q^{\mu}(s,a) |_{a = \mu_{\theta}(s)} ds \\ &= \mathbb{E}_{s \sim \rho^{\mu}} [\triangledown_{\theta} \mu_{\theta}(s) \triangledown_a Q^\mu(s,a) |_{a = \mu_{\theta}(s)} ] \end{split}$$

### SARSA critic

- SARSA critic, on policy actor critic
  - 단점
    - deterministic policy에 의해 행동하면 exploration이 잘 되지 않기 때문에 sub-optimal에 빠지기 쉽다.
  - 목적
    - 환경에서 충분한 noise를 제공하여 exploration 시킬 수 있다면 좋은 결과를 얻을 수 있음.
- update formula
  - \\(Q(s_{t},a_{t}) \leftarrow Q(s_{t},a_{t}) + \alpha(r_{t} + \gamma Q(s_{t+1},a_{t+1}) - Q(s_{t},a_{t}))\\)

- Algorithm

### Q-learning, off-policy actor-critic

- stochastic behavior policy \\(\beta(a|s)\\)에 의해 생성된 경로로 부터 deterministic target policy \\(\mu_{\theta}(s)\\)를 학습하는 off-policy actor-critic

- Q-learning
  - \\(Q(s_{t},a_{t}) \leftarrow Q(s_{t},a_{t}) + \alpha(r_{t} + \gamma \max\limits_{a}Q(s_{t+1},a) - Q(s_{t},a_{t}))\\)

### compatible function approximation 및 gradient temporal-difference learning을 이용한 actor-critic